In [1]:
import pandas as pd

In [ ]:
import pandas as pd
from seq2atac.stable import read_pickle, write_pickle
from pybedtools import BedTool
import scipy.stats

### Get differential peaks and get nearest gene

In [ ]:
!rm DatabaseExtract_v_1.01.csv
!wget https://humantfs.ccbr.utoronto.ca/download/v_1.01/DatabaseExtract_v_1.01.csv ### Downloading a curated list of TFs

In [4]:
gbm_45b_peak_file = "./GBM_Clone_Peaks/GBM45_cloneB_peaks.csv" ### TODO
tss_file = "./tss_tcga.csv" ### TODO
gbm45b_uncleaned_motifs_path = "./GBM_Clone_Uncleaned_Motifs/vierstra_Archetype_GBM45_cloneBagg.csv.bed" ### TODO
gbm45_cleaned_motifs_path = "./GBM_Clone_Cleaned_Motifs/ismshap_gbm_subclone_vierstrav1_GBM45_cloneB.csv" ### TODO

In [ ]:
df_45b_peaks = pd.read_csv(gbm_45b_peak_file)[["seqnames","start","end"]]
df_45b_peaks["summit"] = (df_45b_peaks["start"] + df_45b_peaks["end"])//2
df_45b_peaks["summit_1"] = df_45b_peaks["summit"] + 1
df_45b_peaks["peak_id"] = df_45b_peaks.index
df_45b_peaks


In [6]:
peaksize=500
df_45b_peaks["start"] = df_45b_peaks["start"] + (500-peaksize)//2
df_45b_peaks["end"] = df_45b_peaks["start"] + peaksize

In [7]:
peak_bed = BedTool.from_dataframe(df_45b_peaks[["seqnames","summit","summit_1","peak_id"]])
tss_df = pd.read_csv(tss_file)
tss_bed = BedTool.from_dataframe(tss_df)

In [ ]:
peak_annotated = peak_bed.sort().closest(tss_bed.sort(), d=True, t="first").to_dataframe(names=["seqnames","summit","summit_1","peak_id","chr","tss","tss_1","gene","distance_to_tss"])
peak_annotated

In [ ]:
df_45b_peaks_merged = df_45b_peaks.merge(peak_annotated, how="left", on=["seqnames","summit","summit_1","peak_id"])
assert len(df_45b_peaks_merged) == len(df_45b_peaks)
df_45b_peaks_merged

### Read all cleaned motifs

In [ ]:
gbm45b_allmotifs = pd.read_csv(gbm45b_uncleaned_motifs_path)
gbm45b_allmotifs

In [ ]:
gbm45b_motifs = pd.read_csv(gbm45_cleaned_motifs_path)
print(len(gbm45b_motifs))
gbm45b_motifs = gbm45b_motifs.merge(gbm45b_allmotifs)
gbm45b_motifs

In [ ]:
gbm45b_motifs = gbm45b_motifs[["seqnames","start","end","group_name","individual_match","es"]]
gbm45b_motifs

### Get all cleaned motif instances overlapping differential peaks

In [ ]:
motif_bed = BedTool.from_dataframe(gbm45b_motifs)
peaks_bed = BedTool.from_dataframe(df_45b_peaks_merged[["seqnames","start","end"]])

motif_bed_intersected = motif_bed.intersect(peaks_bed, wa=True, wb=True).to_dataframe(names=["motif_chr","motif_start","motif_end","group_name","individual_match","es","seqnames","start","end"]).merge(df_45b_peaks_merged)
motif_bed_intersected

In [14]:
### Subset to those instances whose archetype contains a Chr6 TF

In [ ]:
all_tfs = pd.read_csv("DatabaseExtract_v_1.01.csv")
display(all_tfs)
tss_df = pd.read_csv(tss_file)
display(tss_df)

In [16]:
tf_chromosome = "chr6"

In [17]:
chr6tfs = sorted(list(tss_df[(tss_df["gene"].isin(all_tfs["HGNC symbol"])) & (tss_df["chr"]==tf_chromosome)]["gene"].unique()))
print(len(chr6tfs))
chr6tfs

136


['AGER',
 'ARID1B',
 'ATF6B',
 'BACH2',
 'BCLAF1',
 'CASP8AP2',
 'CDC5L',
 'CDKN1A',
 'CITED2',
 'DAXX',
 'DEK',
 'DSP',
 'E2F3',
 'EDN1',
 'ESR1',
 'ETV7',
 'EXOC2',
 'FOXC1',
 'FOXF2',
 'FOXO3',
 'FOXP4',
 'FOXQ1',
 'GCM1',
 'GCM2',
 'GTF2H4',
 'HDAC2',
 'HEY2',
 'HIVEP1',
 'HIVEP2',
 'HLA-DQB1',
 'HLA-DQB2',
 'HLA-DRB1',
 'HLA-DRB5',
 'HMGA1',
 'HMGN3',
 'HSF2',
 'HSPA1A',
 'HSPA1B',
 'ID4',
 'IRF4',
 'JARID2',
 'L3MBTL3',
 'LIN28B',
 'MAP3K7',
 'MAPK14',
 'MDFI',
 'MED23',
 'MYB',
 'NCOA7',
 'NFKBIL1',
 'NFYA',
 'NR2E1',
 'OLIG3',
 'PBX2',
 'PGBD1',
 'PHF1',
 'PIM1',
 'PKHD1',
 'PLAGL1',
 'POLH',
 'POU3F2',
 'POU5F1',
 'PPARD',
 'PPP1R10',
 'PRDM1',
 'PRDM13',
 'PRIM2',
 'PRKN',
 'PRR3',
 'RFX6',
 'RING1',
 'RIPK1',
 'RREB1',
 'RUNX2',
 'RXRB',
 'SCML4',
 'SGK1',
 'SHPRH',
 'SIM1',
 'SNRPC',
 'SOGA3',
 'SOX4',
 'SPDEF',
 'SRF',
 'TAB2',
 'TAF11',
 'TAF8',
 'TBP',
 'TBPL1',
 'TBX18',
 'TCF19',
 'TCF21',
 'TDP2',
 'TEAD3',
 'TFAP2A',
 'TFAP2B',
 'TFAP2D',
 'TFEB',
 'TNF',
 'TNFAIP3',
 'TRERF1',
 'TRIM15',
 'TRIM26',
 'TRIM27',
 'TRIM31',
 'TRIM38',
 'TRIM40',
 'TULP4',
 'UFL1',
 'VEGFA',
 'VGLL2',
 'ZBED9',
 'ZBTB12',
 'ZBTB2',
 'ZBTB22',
 'ZBTB24',
 'ZBTB9',
 'ZFP57',
 'ZKSCAN3',
 'ZKSCAN4',
 'ZKSCAN8',
 'ZNF165',
 'ZNF184',
 'ZNF292',
 'ZNF311',
 'ZNF318',
 'ZNF322',
 'ZNF391',
 'ZNF451',
 'ZNF76',
 'ZSCAN12',
 'ZSCAN16',
 'ZSCAN23',
 'ZSCAN26',
 'ZSCAN31',
 'ZSCAN9']

In [ ]:
all_individual_motif_names = list(motif_bed_intersected["individual_match"].unique())
all_individual_motif_names

In [19]:
chr6_individual_names = []
for gene in chr6tfs:
    for ind_name in all_individual_motif_names:
        
        if gene.upper() in ind_name.upper():
            
            chr6_individual_names.append(ind_name)
            
len(chr6_individual_names)

108

In [ ]:
chr6_individual_names

In [21]:
indiviual_name_to_archetype_dict = dict(zip(motif_bed_intersected["individual_match"],
                                            motif_bed_intersected["group_name"]))

In [22]:
chr6_tf_group_names = []

for ind_name in chr6_individual_names:
    
    chr6_tf_group_names.append(indiviual_name_to_archetype_dict[ind_name])

chr6_tf_group_names = list(set(chr6_tf_group_names))
print(chr6_tf_group_names)
sorted(chr6_tf_group_names)

['POU/2', 'RUNX/1', 'OCT4+SOX2', 'E2F/1', 'KLF/SP/2', 'ETS/1', 'MYB/3', 'HD/12', 'MYB/5', 'ZNF143', 'FOX/7', 'RFX/3', 'FOX/5', 'Ebox/CACGTG/1', 'NFY', 'TFAP2/2', 'FOX/2', 'HD/16', 'Ebox/CATATG', 'RUNX/2', 'PRDM1', 'IRF/2', 'ZNF57', 'Ebox/CACCTG', 'NR/14', 'SPDEF/1', 'HSF', 'Ebox/CAGCTG', 'TFAP2/1', 'TATA', 'MYB/1', 'NR/17', 'SOX/4', 'NR/1', 'SOX/1', 'IRF/1', 'SRF', 'TEAD', 'FOX/6', 'POU/3', 'NR/16', 'MYB/2', 'MYB/4', 'FOX/4', 'POU/1', 'GCM', 'AP1/2', 'SPDEF/2', 'E2F/2']


['AP1/2',
 'E2F/1',
 'E2F/2',
 'ETS/1',
 'Ebox/CACCTG',
 'Ebox/CACGTG/1',
 'Ebox/CAGCTG',
 'Ebox/CATATG',
 'FOX/2',
 'FOX/4',
 'FOX/5',
 'FOX/6',
 'FOX/7',
 'GCM',
 'HD/12',
 'HD/16',
 'HSF',
 'IRF/1',
 'IRF/2',
 'KLF/SP/2',
 'MYB/1',
 'MYB/2',
 'MYB/3',
 'MYB/4',
 'MYB/5',
 'NFY',
 'NR/1',
 'NR/14',
 'NR/16',
 'NR/17',
 'OCT4+SOX2',
 'POU/1',
 'POU/2',
 'POU/3',
 'PRDM1',
 'RFX/3',
 'RUNX/1',
 'RUNX/2',
 'SOX/1',
 'SOX/4',
 'SPDEF/1',
 'SPDEF/2',
 'SRF',
 'TATA',
 'TEAD',
 'TFAP2/1',
 'TFAP2/2',
 'ZNF143',
 'ZNF57']

In [ ]:
motif_bed_intersected = motif_bed_intersected[(motif_bed_intersected["group_name"].isin(chr6_tf_group_names))]
motif_bed_intersected


In [ ]:
motif_bed_intersected = motif_bed_intersected.drop_duplicates(["peak_id"])
motif_bed_intersected

In [ ]:
motif_bed_intersected["tf_not"] = motif_bed_intersected["gene"].apply(lambda g : int(g in all_tfs["HGNC symbol"].tolist()))
motif_bed_intersected

In [26]:
l1 = len(motif_bed_intersected[motif_bed_intersected["tf_not"]==1])
l2 = len(motif_bed_intersected[motif_bed_intersected["tf_not"]==0])
l3 = len(all_tfs)
total_numgenes = 19619
l4 = total_numgenes-len(all_tfs)

In [27]:
contingency = [[l1,l2],[l3,l4]]
contingency

[[11645, 53809], [2765, 16854]]

In [28]:
scipy.stats.fisher_exact(contingency)

(1.3191446660324273, 8.52289537064899e-35)